example logger code.

In [2]:
from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate an API token from the "API Tokens Tab" in the UI
token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
org = "Arbel"
bucket = "speedtest_log"

with InfluxDBClient(url="http://rp1.local:8086", token=token, org=org) as client:
  write_api = client.write_api(write_options=SYNCHRONOUS)

data = "mem,host=host1 used_percent=1234"
write_api.write(bucket, org, data)


In [ ]:
from datetime import datetime
import socket
import os
import re
import subprocess
import time

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate an API token from the "API Tokens Tab" in the UI
influxdb_key = open("../data/influxdb.key", 'r')
token = influxdb_key.read()
org = "Arbel"
bucket = "speedtest_log"

with InfluxDBClient(url="http://localhost:8086", token=token, org=org) as client:
  write_api = client.write_api(write_options=SYNCHRONOUS)

response = subprocess.Popen('/usr/bin/speedtest -s 31817 --accept-license --accept-gdpr',
                            shell=True, stdout=subprocess.PIPE).stdout.read().decode('utf-8')

ping = re.search('Latency:\s+(.*?)\s', response, re.MULTILINE)
download = re.search('Download:\s+(.*?)\s', response, re.MULTILINE)
upload = re.search('Upload:\s+(.*?)\s', response, re.MULTILINE)
jitter = re.search('Latency:.*?jitter:\s+(.*?)ms', response, re.MULTILINE)

ping = ping.group(1)
download = download.group(1)
upload = upload.group(1)
jitter = jitter.group(1)

speed_data = [
    {
        "measurement": "internet_speed",
        "tags": {
            "host": "rp1.local"
        },
        "fields": {
            "download": float(download),
            "upload": float(upload),
            "ping": float(ping),
            "jitter": float(jitter)
        }
    }
]

#print(speed_data)
write_api.write(bucket, org, speed_data)


"c:/Program Files/WindowsApps/PythonSoftwareFoundation.Python.3.10_3.10.1776.0_x64__qbz5n2kfra8p0/python3.10.exe" -m pip install ipykernel -U --user --force-reinstall